In [ ]:
# Gets decription of books scraped in 1_get_romantasy_data from google books api! Also cleans this data to be ready for modeling.  

In [58]:
import json
from urllib.request import urlopen
import pandas as pd
import numpy as np
from simplejson import JSONDecodeError
import time
from urllib.error import HTTPError

In [157]:
df= pd.read_csv("fantasy_romance_titles_authors.csv")
df

,Title,Author,Rating,Count of Ratings,img
0,A Court of Thorns and Roses,Sarah J. Maas,4.18,"3,459,145",https://i.gr-assets.com/images/S/compressed.ph...
1,A Court of Mist and Fury,Sarah J. Maas,4.65,"2,625,974",https://i.gr-assets.com/images/S/compressed.ph...
2,Fourth Wing,Rebecca Yarros,4.58,"2,508,672",https://i.gr-assets.com/images/S/compressed.ph...
3,A Court of Wings and Ruin,Sarah J. Maas,4.47,"2,194,491",https://i.gr-assets.com/images/S/compressed.ph...
4,From Blood and Ash,Jennifer L. Armentrout,4.22,"708,676",https://i.gr-assets.com/images/S/compressed.ph...
...,...,...,...,...,...
1245,Blaze of Memory,Nalini Singh,3.96,"24,997",https://i.gr-assets.com/images/S/compressed.ph...
1246,Split or Swallow,Lindsay Straube,3.83,"5,662",https://i.gr-assets.com/images/S/compressed.ph...
1247,Autumn of the Grimoire,J.L. Vampa,4.05,"3,185",https://i.gr-assets.com/images/S/compressed.ph...
1248,Curious Tides,Pascale Lacelle,3.81,"10,090",https://i.gr-assets.com/images/S/compressed.ph...


In [173]:
# pull list of api links to loop over -- this is so we can check if we have already pulled data for one of the links and don't do it twice! 
api_list_full= []
title_list=[]
author_list=[]
rating_list=[]
image_link_list=[]
count_of_rating_list=[] 
for i in range(0, len(df)): 
    title= df.iloc[i]["Title"]
    title = title.replace("’", '')
    title= title.replace('ü', 'u')
    author= df.iloc[i]["Author"]
    author= author.replace('.', '')
    if 'é' in author: 
        author = author.replace('é', 'e')
    if 'ü' in author: 
        author = author.replace('ü', 'u')
    if "ñ" in author: 
        author = author.replace('ñ', 'n')
    api = "https://www.googleapis.com/books/v1/volumes?q=intitle:" + str(title) +"+inauthor:" + str(author) +"&maxResults=1"
    api= api.replace(" ", "%20")
    api_list_full.append(api)
    title_list.append(title)
    author_list.append(author)
    rating_list.append(df.iloc[i]["Rating"])
    count_of_rating_list.append(df.iloc[i]["Count of Ratings"])
    image_link_list.append(df.iloc[i]["img"])

In [162]:
# now get book data from api link! 

def get_book_data(api, title, author, avg_rating, count_of_ratings, image_link): 
    
    # send a request and get a JSON response
    try: 
        resp = urlopen(api)
    except UnicodeEncodeError: 
        print(author)
        print(api)
    # parse JSON into Python as a dictionary
    try: 
        book = json.load(resp)
    except JSONDecodeError:  # if we get a weird decode error, skip and print api! 
        print("json: " + str(api))
        pass 

    if book["totalItems"] != 0: 
        book= book['items'][0] #-- not an index, just json quirk
        try: 
            publisher= book['volumeInfo']['publisher']
        except KeyError: 
            publisher = "N/A"
        try: 
            published_date= book['volumeInfo']['publishedDate'] 
        except KeyError: 
            published_date = "N/A"
        try: 
            description= book['volumeInfo']['description']
        except KeyError: 
            description= 'N/A'
            print("descr: " + str(api))
        try: 
            page_count=book['volumeInfo']['pageCount']
        except KeyError: 
            page_count= 'N/A'
            print("page_count: " + str(api))
        try: 
            isbn= book['volumeInfo']['industryIdentifiers'][0]['identifier']
        except KeyError: 
            isbn= 'N/A'
            print("isbn: " + str(api))
        try: 
            maturity_rating=book['volumeInfo']['maturityRating']
        except KeyError: 
            maturity_rating="N/A"
        #try: 
        #    image_link= book['volumeInfo']['imageLinks']['thumbnail'] 
       # except KeyError: 
       #     image_link= 'N/A'
       #     print("image_link: " + str(api))
        
        return ({"api_link" : api, "title" : title, "author": author, "avg_rating": avg_rating, "count_of_ratings": count_of_ratings,
                 "publisher": publisher, "published_date": published_date, "description": description, "page_count": page_count, 
                 "isbn": isbn, "maturity_rating": maturity_rating, "image_link": image_link})


In [159]:
#example of get_book_data in action
i= 0
get_book_data(api_list_full[i], title_list[i], author_list[i], rating_list[i], count_of_rating_list[i])

{'api_link': 'https://www.googleapis.com/books/v1/volumes?q=intitle:A%20Court%20of%20Thorns%20and%20Roses+inauthor:Sarah%20J%20Maas&maxResults=1',
 'title': 'A Court of Thorns and Roses',
 'author': 'Sarah J Maas',
 'avg_rating': np.float64(4.18),
 'count_of_ratings': '3,459,145',
 'publisher': 'Bloomsbury Publishing',
 'published_date': '2020-10-01',
 'description': "The first instalment of the GLOBAL PHENOMENON and TikTok sensation, from multi-million selling and #1 Sunday Times bestselling author Sarah J. Maas Maas has established herself as a fantasy fiction titan – Time Harry Potter magic, Taylor Swift sass, Fifty Shades-level athleticism – The Sunday Times With bits of Buffy, Game of Thrones and Outlander, this is a glorious series of total joy – Stylist Spiced with slick plotting and atmospheric world-building ... a page-turning delight – Guardian ****** Feyre is a huntress, but when she kills what she thinks is a wolf in the woods, a terrifying creature arrives to demand retrib

In [160]:
# now do it for all the books! keep separate so can iterate and debug without making all the requests again 
full_book_dictionary={}

In [176]:
for i in range(len(api_list_full)): 
    if api_list_full[i] not in full_book_dictionary.keys(): 
        try: 
            full_book_dictionary[api_list_full[i]]= get_book_data(api_list_full[i], title_list[i], author_list[i], rating_list[i], count_of_rating_list[i], image_link_list[i])
        except HTTPError: # if we've passed too many requests, wait for two minutes
            time.sleep(30)
            full_book_dictionary[api_list_full[i]]= get_book_data(api_list_full[i], title_list[i], author_list[i], rating_list[i], count_of_rating_list[i], image_link_list[i])
    else: 
        pass # we've already got the data for it! this helps cut down on duplicate requests :) 

In [177]:
len(full_book_dictionary) # -- those that are missing did not match to google books! 

1197

In [22]:
full_book_dictionary.keys()

dict_keys(['https://www.googleapis.com/books/v1/volumes?q=intitle:A%20Court%20of%20Thorns%20and%20Roses+inauthor:Sarah%20J%20Maas&maxResults=1', 'https://www.googleapis.com/books/v1/volumes?q=intitle:A%20Court%20of%20Mist%20and%20Fury+inauthor:Sarah%20J%20Maas&maxResults=1', 'https://www.googleapis.com/books/v1/volumes?q=intitle:Fourth%20Wing+inauthor:Rebecca%20Yarros&maxResults=1', 'https://www.googleapis.com/books/v1/volumes?q=intitle:A%20Court%20of%20Wings%20and%20Ruin+inauthor:Sarah%20J%20Maas&maxResults=1', 'https://www.googleapis.com/books/v1/volumes?q=intitle:From%20Blood%20and%20Ash+inauthor:Jennifer%20L%20Armentrout&maxResults=1', 'https://www.googleapis.com/books/v1/volumes?q=intitle:A%20Court%20of%20Silver%20Flames+inauthor:Sarah%20J%20Maas&maxResults=1', 'https://www.googleapis.com/books/v1/volumes?q=intitle:A%20Court%20of%20Frost%20and%20Starlight+inauthor:Sarah%20J%20Maas&maxResults=1', 'https://www.googleapis.com/books/v1/volumes?q=intitle:The%20Serpent%20and%20the%20Win

In [50]:
val= 'https://www.googleapis.com/books/v1/volumes?q=intitle:A%20Court%20of%20Thorns%20and%20Roses+inauthor:Sarah%20J%20Maas&maxResults=1'

pd.json_normalize(full_book_dictionary[val]).iloc[0]

api_link            https://www.googleapis.com/books/v1/volumes?q=...
title                                     A Court of Thorns and Roses
author                                                   Sarah J Maas
avg_rating                                                       4.18
count_of_ratings                                            3,459,145
publisher                                       Bloomsbury Publishing
published_date                                             2020-10-01
description         The first instalment of the GLOBAL PHENOMENON ...
page_count                                                        451
isbn                                                    9781526634245
maturity_rating                                            NOT_MATURE
image_link          https://i.gr-assets.com/images/S/compressed.ph...
Name: 0, dtype: object

In [51]:
full_list= [] 
for key in full_book_dictionary.keys(): 
    val= full_book_dictionary[key]
    if type(val) == type(None): 
        pass 
    else:
       full_list.append(list(pd.json_normalize(val).iloc[0]))

In [97]:
df2 = pd.DataFrame(full_list, columns = ['api', 'Title', 'Author', 'Rating', 'Count of Ratings', 'publisher', 'published_date', 'description', 'page_count', 'isbn', 'maturity_rating', 'image_link']) 

In [55]:
df2.to_csv("romantasy_data.csv", index=False)

In [191]:
#with open('romantasy_data.json', 'w') as f:
#    json.dump(full_book_dictionary, f)

In [12]:
# Open and read the JSON file
#with open('romantasy_data.json', 'r') as file:
#    full_book_dictionary = json.load(file)

## clean romantasy data! 

In [ ]:
# NLP for title? 
# NLP for author? 
# average rating (scale between 0 and 1)
# count of ratings -- convert to categorical? i.e. max 
# publisher -- categorical (?) EDA for how many 
# published_date -- split into year groupings? 
# decription -- NLP for sure 
# page_count -- categorical 
# maturity_rating -- binary 
# image_link -- use within CNN?

In [ ]:
# also do NA checks!!! + fill in! 

In [98]:
df2.dtypes

api                  object
Title                object
Author               object
Rating              float64
Count of Ratings     object
publisher            object
published_date       object
description          object
page_count           object
isbn                 object
maturity_rating      object
image_link           object
dtype: object

In [152]:
df2.replace(to_replace = "N/A", value = np.nan, inplace = True) # replace N/A with np.nan
nan_counts = df2.isna().sum()
nan_counts

api                  0
Title                0
Author               0
Rating               0
Count of Ratings     0
publisher            0
published_date       4
description         46
page_count           0
isbn                13
maturity_rating      0
image_link           0
published_year       0
dtype: int64

In [109]:
# convert to datetime
df2['published_date']= pd.to_datetime(df2['published_date'], format='mixed')
df2['published_date']= pd.to_datetime(df2['published_date'], utc=True)

/var/folders/09/l3m63nj90fg1l2jshjp_hd5w0000gn/T/ipykernel_28864/66946447.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df2['published_date']= pd.to_datetime(df2['published_date'], format='mixed')


In [137]:
df2['published_year'] = df2['published_date'].dt.year
df2['published_year']= df2['published_year'].fillna(df2['published_year'].mean())

In [139]:
df2['page_count']= df2['page_count'].fillna(df2['page_count'].mean()) # fill NA's with mean for page count! 


In [244]:
def get_cat_page_count(page_count): 
    if page_count < 300: 
       return "<300"
    if page_count > 299 and page_count <450: 
        return "300-450"
    if page_count >=450: 
        return "450+"


In [245]:
df2['cleaned_page_count_v1']= df2['page_count'].apply(get_cat_page_count)
df2['cleaned_page_count_v1'].value_counts()


cleaned_page_count_v1
300-450    407
<300       403
450+       256
Name: count, dtype: int64

In [250]:
df2['count_of_ratings']= df2['Count of Ratings'].str.replace(",", "")
df2['count_of_ratings']= df2['count_of_ratings'].astype(int)


In [269]:
def get_cat_ratings(rating): 
    if rating > 130000: 
       return 1
    elif rating > 75000: 
        return 2 
    elif rating > 50000: 
        return 3
    elif rating > 25000: 
            return 4
    elif rating > 10000: 
            return 5
    else:
         return 6


In [270]:
df2['cleaned_count_ratings_v1']= df2['count_of_ratings'].apply(get_cat_ratings)
df2['cleaned_count_ratings_v1'].value_counts()

cleaned_count_ratings_v1
5    230
1    228
6    204
4    179
2    123
3    102
Name: count, dtype: int64

In [273]:
def get_mature(mature): 
    if mature == "MATURE": 
        return 1
    else: 
        return 0 

df2['mature']= df2['maturity_rating'].apply(get_mature)
df2['mature'].value_counts()



mature
0    1010
1      56
Name: count, dtype: int64

In [142]:
def get_cleaned_publisher(publisher_name): 
    publisher_name = publisher_name.lower()
    if "bloomsbury" in publisher_name: 
        cleaned_publisher= "bloomsbury"
    elif 'harper' in publisher_name: 
        cleaned_publisher= "harper collins"
    elif 'entangled' in publisher_name: 
        cleaned_publisher= 'entangled'
    elif 'kensington' in publisher_name: 
        cleaned_publisher = 'kensington'
    elif 'penguin' in publisher_name or 'random house' in publisher_name: 
        cleaned_publisher = 'random house'
    elif 'martin' in publisher_name: 
        cleaned_publisher='martin'
    elif 'brown books' in publisher_name: 
        cleaned_publisher='little brown books'
    else: 
        cleaned_publisher ='other'
    return cleaned_publisher


In [150]:
# add back in string value! 
df2['publisher'].replace(to_replace = np.nan, value = "N/A", inplace = True) # replace N/A with np.nan

df2['cleaned_publisher']= df2['publisher'].apply(get_cleaned_publisher)

df2['cleaned_publisher'].value_counts()

publisher
other                 778
random house          128
harper collins         78
bloomsbury             24
little brown books     22
entangled              19
kensington              9
martin                  8
Name: count, dtype: int64

In [151]:
# Bloomsberg 
# Collins Harper Collings 
# Cambridge University Press
# Entangled 
# Kensington
# Penguin Random House 
# St Martin's Press
# Little brown books 
# Other 
#np.unique(df2['publisher'])

In [170]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk


In [173]:
## now onto cleaning descriptions! 

def clean_text(text):
    # Lowercase conversion
    text = text.lower()
    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

cleaned_description = clean_text()


thrilling adventure touch mystery published


In [176]:
# add back in string value! 
df2['description'].replace(to_replace = np.nan, value = "N/A", inplace = True) # replace N/A with np.nan

df2['cleaned_description'] = df2['description'].apply(clean_text)

In [193]:
df2['cleaned_author'] = df2['Author'].apply(clean_text)
df2['cleaned_title'] = df2['Title'].apply(clean_text)

In [180]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [194]:

corpus= list(df2['cleaned_description'])

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
word_index_description = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(corpus)

# Pad sequences
max_length = 256  # Fixed length for CNN input
padded_sequences_description = pad_sequences(sequences, padding='post',  maxlen=max_length) 


In [228]:
corpus_author= list(df2['cleaned_author'])

# Tokenize the text
tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(corpus_author)
word_index_author = tokenizer2.texts_to_sequences
sequences2 = tokenizer2.texts_to_sequences(corpus_author)

# Pad sequences
padded_sequences_author = pad_sequences(sequences2, padding='post') #,  maxlen=max_length) 


In [ ]:
corpus_title= list(df2['cleaned_title'])

# Tokenize the text
tokenizer3 = Tokenizer()
tokenizer3.fit_on_texts(corpus_title)
word_index_title = tokenizer3.texts_to_sequences
sequences3 = tokenizer3.texts_to_sequences(corpus_title)

# Pad sequences
padded_sequences_title = pad_sequences(sequences3, padding='post') #,  maxlen=max_length) 


In [ ]:
# here are the features we have so far! last *big* hurdle would be to include image data! 
# ['cleaned_title', 'cleaned_author' , 'cleaned_description', 'cleaned_publisher', 
# 'cleaned_page_count_v1', 'cleaned_count_ratings_v1', 'published_year', 'mature'] 

In [279]:
df2[['cleaned_title', 'cleaned_author' , 'Rating', "Count of Ratings", 'cleaned_description', 'cleaned_publisher', 
     'cleaned_page_count_v1', 'cleaned_count_ratings_v1', 'published_year', 'mature']]

,cleaned_title,cleaned_author,Rating,Count of Ratings,cleaned_description,cleaned_publisher,cleaned_page_count_v1,cleaned_count_ratings_v1,published_year,mature
0,court thorn rose,sarah j maas,4.18,"3,459,145",first instalment global phenomenon tiktok sens...,bloomsbury,450+,1,2020.0,0
1,court mist fury,sarah j maas,4.65,"2,625,974",new york time bestselling sequel sarah j maass...,bloomsbury,450+,1,2017.0,0
2,fourth wing,rebecca yarros,4.58,"2,508,672",war grows deadly violet sorrengail join elite ...,entangled,<300,1,2023.0,0
3,court wing ruin,sarah j maas,4.47,"2,194,491",feyre must decide trust among high lord turn a...,bloomsbury,450+,1,2017.0,0
4,blood ash,jennifer l armentrout,4.22,"708,676",captivating actionpacked blood ash sexy addict...,other,300-450,1,2020.0,0
...,...,...,...,...,...,...,...,...,...,...
1061,scorpio race,maggie stiefvater,4.10,"111,812",every november scorpio race run beneath chalk ...,other,<300,2,2022.0,0
1062,blaze memory,nalini singh,3.96,"24,997",nalini singh return psychangeling world woman ...,other,300-450,5,2011.0,0
1063,split swallow,lindsay straube,3.83,"5,662",internet sensation amazon bestseller lindsay s...,other,450+,6,2025.0,0
1064,autumn grimoire,jl vampa,4.05,"3,185",witch cast spell others slay king mysterious g...,other,300-450,6,2022.0,0


In [282]:
df2[['api', 'Title', 'cleaned_title', 'cleaned_author' , 'Rating', "Count of Ratings", 'cleaned_description', 'cleaned_publisher', 
     'cleaned_page_count_v1', 'cleaned_count_ratings_v1', 'published_year', 'mature']].to_csv("cleaned_romantasy_data.csv", index=False)
     

In [ ]:
## old version -- didn't use dict so would make new requests. 
for i in range(0, len(df)): 
    title= df.iloc[i]["Title"]
    author= df.iloc[i]["Author"]
    author= author.replace('.', '')
    api = "https://www.googleapis.com/books/v1/volumes?q=intitle:" + str(title) +"+inauthor:" + str(author) +"&maxResults=1"
    api= api.replace(" ", "%20")
    api.append(api_list_full)

    # send a request and get a JSON response
    try: 
        resp = urlopen(api)
    except UnicodeEncodeError: 
        pass
    # parse JSON into Python as a dictionary
    try: 
        book = json.load(resp)
    except JSONDecodeError: 
        print("json: " + str(api))
        pass 

    if book["totalItems"] != 0: 
        book= book['items'][0] #-- not an index, just json quirk
        try: 
            publisher= book['volumeInfo']['publisher']
        except KeyError: 
            publisher = "N/A"
        try: 
            published_date= book['volumeInfo']['publishedDate'] 
        except KeyError: 
            publisher = "N/A"
        try: 
            description= book['volumeInfo']['description']
        except KeyError: 
            description= 'N/A'
            print("descr: " + str(api))
        try: 
            page_count=book['volumeInfo']['pageCount']
        except KeyError: 
            page_count= 'N/A'
            print("page_count: " + str(api))
        try: 
            isbn= book['volumeInfo']['industryIdentifiers'][0]['identifier']
        except KeyError: 
            isbn= 'N/A'
            print("isbn: " + str(api))
        try: 
            maturity_rating=book['volumeInfo']['maturityRating']
        except KeyError: 
            maturity_rating="N/A"
        try: 
            image_link= book['volumeInfo']['imageLinks']['thumbnail'] 
        except KeyError: 
            image_link= 'N/A'
            print("image_link: " + str(api))
        
        book_data_full.append([api, title, df.iloc[i]["Author"], df.iloc[i]["Rating"], df.iloc[i]["Count of Ratings"], publisher, published_date, description, page_count, isbn, maturity_rating, image_link])   
    else: 
        print(api)
        pass
 

https://www.googleapis.com/books/v1/volumes?q=intitle:Radiance+inauthor:Grace%20Draven&maxResults=1
descr: https://www.googleapis.com/books/v1/volumes?q=intitle:A%20Shadow%20in%20the%20Ember+inauthor:Jennifer%20L%20Armentrout&maxResults=1
image_link: https://www.googleapis.com/books/v1/volumes?q=intitle:A%20Shadow%20in%20the%20Ember+inauthor:Jennifer%20L%20Armentrout&maxResults=1
https://www.googleapis.com/books/v1/volumes?q=intitle:Master%20of%20Crows+inauthor:Grace%20Draven&maxResults=1
https://www.googleapis.com/books/v1/volumes?q=intitle:Entreat%20Me+inauthor:Grace%20Draven&maxResults=1
descr: https://www.googleapis.com/books/v1/volumes?q=intitle:A%20Kingdom%20of%20Stars%20and%20Shadows+inauthor:Holly%20Renee&maxResults=1
image_link: https://www.googleapis.com/books/v1/volumes?q=intitle:A%20Kingdom%20of%20Stars%20and%20Shadows+inauthor:Holly%20Renee&maxResults=1
page_count: https://www.googleapis.com/books/v1/volumes?q=intitle:Eidolon+inauthor:Grace%20Draven&maxResults=1
https://ww

HTTPError: HTTP Error 429: Too Many Requests